In [1]:
from __future__ import print_function, division

import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob

from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
import time

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []
    i = 0
    for file in glob.glob("jazz_data/*.mid"):
        midi = converter.parse(file)
        i = i + 1
        if i > 5: break
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        print('[train]: Starting...')
        time_start = time.time()
        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Insert saving; The generator image is saved every 500 steps
        save_interval = 500

        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)



            if (epoch + 1) % save_interval == 0:

                # Save the generator model in a periodic basis
                model_name = './jazz_models/bts_' + str(epoch)
                self.generator.save(model_name + ".h5")

                # Generate a midi file in a periodic basis
                self.generate(notes, epoch)


            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [time: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss, time.time() - time_start))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes, epoch)
        self.plot_loss()


        time_end = time.time()
        print('[train]: Ended. Time elapsed: {}'.format(time_end - time_start))
        
    def generate(self, input_notes, epoch):
        # Get pitch names and store in a dictionary
        print('[generate midi]: Starting...')
        time_start = time.time()
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        #print(pitchnames)
        #print(len(pitchnames))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        #print(len(int_to_note))
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        #print(predictions[0])
        pred_notes = []


        old_max = 1
        old_min = -1
        new_max = 41
        new_min = 0
        old_range = old_max - old_min
        new_range = new_max - new_min
        for i, x in enumerate(predictions[0]):
            entry = (((x - old_min)* new_range)/old_range) + new_min
            # if x < 0:
            #     entry = (x+1)*21
            # else:
            #     entry = (x*21) + 21
            pred_notes.append(entry)
        #pred_notes = [int((x+1)*89) for x in predictions[0]]
        #pred_notes = [x*242+242 for x in predictions[0]]
        #print(len(int_to_note))
        #print(pred_notes)
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        

        file_name = './jazz_output/jazz_' + str(epoch)
        create_midi(pred_notes, file_name)
        time_end = time.time()
        print('[generate midi]: Ended. Time elapsed: {}'.format(time_end - time_start))

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('./jazz_output/GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

if __name__ == '__main__':
  gan = GAN(rows=100)    
  gan.train(epochs=2500, batch_size=32, sample_interval=1)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dens

/home/dlstudent02/anaconda3/envs/coe197/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.695659, acc.: 46.88%] [G loss: 0.690256] [time: 7.147846]
1 [D loss: 0.681848, acc.: 62.50%] [G loss: 0.691927] [time: 7.262049]
2 [D loss: 0.660362, acc.: 65.62%] [G loss: 0.696626] [time: 7.370209]
3 [D loss: 0.623800, acc.: 76.56%] [G loss: 0.716360] [time: 7.482069]
4 [D loss: 0.582547, acc.: 75.00%] [G loss: 0.888905] [time: 7.586531]
5 [D loss: 0.365907, acc.: 90.62%] [G loss: 1.559765] [time: 7.689550]
6 [D loss: 0.128344, acc.: 96.88%] [G loss: 6.562968] [time: 7.790627]
7 [D loss: 0.143568, acc.: 96.88%] [G loss: 8.016457] [time: 7.893084]
8 [D loss: 0.194340, acc.: 96.88%] [G loss: 2.643971] [time: 7.993237]
9 [D loss: 0.170156, acc.: 93.75%] [G loss: 7.267745] [time: 8.094290]
10 [D loss: 0.081397, acc.: 96.88%] [G loss: 11.054043] [time: 8.195860]
11 [D loss: 0.669240, acc.: 81.25%] [G loss: 1.621484] [time: 8.300367]
12 [D loss: 0.304298, acc.: 93.75%] [G loss: 1.772068] [time: 8.404845]
13 [D loss: 0.306335, acc.: 89.06%] [G loss: 3.426913] [time: 8.508658]
1

114 [D loss: 0.446747, acc.: 82.81%] [G loss: 1.752255] [time: 18.982778]
115 [D loss: 0.375653, acc.: 84.38%] [G loss: 1.921044] [time: 19.085795]
116 [D loss: 0.355782, acc.: 82.81%] [G loss: 2.511790] [time: 19.188201]
117 [D loss: 0.564244, acc.: 76.56%] [G loss: 1.457940] [time: 19.291498]
118 [D loss: 0.538076, acc.: 79.69%] [G loss: 1.285463] [time: 19.393447]
119 [D loss: 0.534626, acc.: 79.69%] [G loss: 1.424088] [time: 19.495860]
120 [D loss: 0.453129, acc.: 82.81%] [G loss: 1.891523] [time: 19.600182]
121 [D loss: 0.270459, acc.: 87.50%] [G loss: 4.371991] [time: 19.703552]
122 [D loss: 0.509218, acc.: 84.38%] [G loss: 1.225153] [time: 19.804389]
123 [D loss: 0.488472, acc.: 78.12%] [G loss: 1.387781] [time: 19.905266]
124 [D loss: 0.303928, acc.: 90.62%] [G loss: 4.973877] [time: 20.006824]
125 [D loss: 0.508744, acc.: 81.25%] [G loss: 1.687824] [time: 20.107497]
126 [D loss: 0.424088, acc.: 82.81%] [G loss: 1.509153] [time: 20.208202]
127 [D loss: 0.471715, acc.: 79.69%] [

...

1976 [D loss: 0.708722, acc.: 40.62%] [G loss: 0.738486] [time: 211.435668]
1977 [D loss: 0.680105, acc.: 65.62%] [G loss: 0.733121] [time: 211.538592]
1978 [D loss: 0.667203, acc.: 62.50%] [G loss: 0.736392] [time: 211.639642]
1979 [D loss: 0.667508, acc.: 68.75%] [G loss: 0.732186] [time: 211.741700]
1980 [D loss: 0.711227, acc.: 42.19%] [G loss: 0.711397] [time: 211.845052]
1981 [D loss: 0.688646, acc.: 53.12%] [G loss: 0.705143] [time: 211.952995]
1982 [D loss: 0.697588, acc.: 51.56%] [G loss: 0.701165] [time: 212.057315]
1983 [D loss: 0.687379, acc.: 54.69%] [G loss: 0.705710] [time: 212.161459]
1984 [D loss: 0.699943, acc.: 43.75%] [G loss: 0.718295] [time: 212.266594]
1985 [D loss: 0.680654, acc.: 59.38%] [G loss: 0.721261] [time: 212.371334]
1986 [D loss: 0.678791, acc.: 62.50%] [G loss: 0.736133] [time: 212.476211]
1987 [D loss: 0.688388, acc.: 48.44%] [G loss: 0.726144] [time: 212.578974]
1988 [D loss: 0.678777, acc.: 54.69%] [G loss: 0.711610] [time: 212.681585]
1989 [D loss

2084 [D loss: 0.694345, acc.: 45.31%] [G loss: 0.724680] [time: 222.754544]
2085 [D loss: 0.687526, acc.: 57.81%] [G loss: 0.713755] [time: 222.858675]
2086 [D loss: 0.708203, acc.: 45.31%] [G loss: 0.715571] [time: 222.960606]
2087 [D loss: 0.700424, acc.: 45.31%] [G loss: 0.712906] [time: 223.066534]
2088 [D loss: 0.686599, acc.: 57.81%] [G loss: 0.720845] [time: 223.172874]
2089 [D loss: 0.697616, acc.: 42.19%] [G loss: 0.699518] [time: 223.275882]
2090 [D loss: 0.694964, acc.: 51.56%] [G loss: 0.705584] [time: 223.382651]
2091 [D loss: 0.680491, acc.: 64.06%] [G loss: 0.714568] [time: 223.487853]
2092 [D loss: 0.683697, acc.: 64.06%] [G loss: 0.714976] [time: 223.591890]
2093 [D loss: 0.683424, acc.: 57.81%] [G loss: 0.707877] [time: 223.695516]
2094 [D loss: 0.694197, acc.: 50.00%] [G loss: 0.715508] [time: 223.798397]
2095 [D loss: 0.685327, acc.: 56.25%] [G loss: 0.713626] [time: 223.902395]
2096 [D loss: 0.687540, acc.: 64.06%] [G loss: 0.709639] [time: 224.006590]
2097 [D loss

2192 [D loss: 0.687015, acc.: 53.12%] [G loss: 0.710572] [time: 233.992212]
2193 [D loss: 0.692053, acc.: 50.00%] [G loss: 0.713715] [time: 234.096400]
2194 [D loss: 0.691608, acc.: 46.88%] [G loss: 0.693636] [time: 234.199754]
2195 [D loss: 0.696094, acc.: 46.88%] [G loss: 0.692541] [time: 234.301643]
2196 [D loss: 0.686950, acc.: 53.12%] [G loss: 0.703305] [time: 234.402745]
2197 [D loss: 0.694735, acc.: 48.44%] [G loss: 0.727965] [time: 234.506921]
2198 [D loss: 0.687350, acc.: 51.56%] [G loss: 0.719613] [time: 234.608431]
2199 [D loss: 0.684112, acc.: 56.25%] [G loss: 0.776162] [time: 234.711869]
2200 [D loss: 0.692505, acc.: 51.56%] [G loss: 0.698415] [time: 234.813073]
2201 [D loss: 0.699901, acc.: 37.50%] [G loss: 0.708525] [time: 234.916005]
2202 [D loss: 0.687458, acc.: 54.69%] [G loss: 0.700550] [time: 235.017077]
2203 [D loss: 0.690456, acc.: 46.88%] [G loss: 0.712512] [time: 235.119272]
2204 [D loss: 0.683833, acc.: 57.81%] [G loss: 0.730394] [time: 235.221208]
2205 [D loss

2300 [D loss: 0.694118, acc.: 54.69%] [G loss: 0.751624] [time: 245.108686]
2301 [D loss: 0.699357, acc.: 43.75%] [G loss: 0.755562] [time: 245.213691]
2302 [D loss: 0.699640, acc.: 51.56%] [G loss: 0.737924] [time: 245.315635]
2303 [D loss: 0.700258, acc.: 39.06%] [G loss: 0.730607] [time: 245.417380]
2304 [D loss: 0.675324, acc.: 60.94%] [G loss: 0.712525] [time: 245.520993]
2305 [D loss: 0.674285, acc.: 43.75%] [G loss: 0.753847] [time: 245.623251]
2306 [D loss: 0.732363, acc.: 43.75%] [G loss: 0.692963] [time: 245.728284]
2307 [D loss: 0.698276, acc.: 50.00%] [G loss: 0.697228] [time: 245.833040]
2308 [D loss: 0.687516, acc.: 57.81%] [G loss: 0.700634] [time: 245.935337]
2309 [D loss: 0.686424, acc.: 56.25%] [G loss: 0.702576] [time: 246.037716]
2310 [D loss: 0.685510, acc.: 59.38%] [G loss: 0.746556] [time: 246.139893]
2311 [D loss: 0.694294, acc.: 51.56%] [G loss: 0.723294] [time: 246.245286]
2312 [D loss: 0.685301, acc.: 56.25%] [G loss: 0.704795] [time: 246.349626]
2313 [D loss

2408 [D loss: 0.662871, acc.: 62.50%] [G loss: 0.739404] [time: 256.223189]
2409 [D loss: 0.685639, acc.: 57.81%] [G loss: 0.736055] [time: 256.326738]
2410 [D loss: 0.684938, acc.: 56.25%] [G loss: 0.710795] [time: 256.425546]
2411 [D loss: 0.687804, acc.: 54.69%] [G loss: 0.740067] [time: 256.530274]
2412 [D loss: 0.692543, acc.: 50.00%] [G loss: 0.698493] [time: 256.632700]
2413 [D loss: 0.692947, acc.: 51.56%] [G loss: 0.739393] [time: 256.737316]
2414 [D loss: 0.671143, acc.: 56.25%] [G loss: 0.728075] [time: 256.839302]
2415 [D loss: 0.671460, acc.: 57.81%] [G loss: 0.744162] [time: 256.943190]
2416 [D loss: 0.677896, acc.: 54.69%] [G loss: 0.724351] [time: 257.046037]
2417 [D loss: 0.677693, acc.: 59.38%] [G loss: 0.721296] [time: 257.148564]
2418 [D loss: 0.634443, acc.: 62.50%] [G loss: 0.767002] [time: 257.252224]
2419 [D loss: 0.660180, acc.: 60.94%] [G loss: 0.792003] [time: 257.355929]
2420 [D loss: 0.753540, acc.: 54.69%] [G loss: 0.755438] [time: 257.459718]
2421 [D loss